In [1]:
%load_ext lab_black
%matplotlib inline
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tabulate

warnings.filterwarnings("ignore")

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
cases_and_controls = pd.read_csv(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/dr_7934_pats.txt",
    sep="|",
)

In [3]:
metadata_path = "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/chimec_mammo_retry.csv"  # output of filter_and_convert.py script
metadata = pd.read_csv(metadata_path)
print("loaded {} cases".format(metadata[metadata.case == True].study_id.nunique()))
print("loaded {} controls".format(metadata[metadata.case == False].study_id.nunique()))
print("loaded {} exams".format(metadata.exam_id.nunique()))

loaded 191 cases
loaded 1839 controls
loaded 12192 exams


In [4]:
df = pd.DataFrame(metadata)
# selecting columns to keep
columns_to_keep = [
    "study_id",
    "exam_id",
    "BurnedInAnnotation",
    "PresentationIntentType",
    "BreastImplantPresent",
    "ContentDate",
    "ContentTime",
    "ImageType",
    "Laterality",
    "Modality",
    "PatientName",
    "SeriesDescription",
    "ImageLaterality",
    "ViewPosition",
    "ProtocolName",
    "Study DateTime",
    "DNA_available",
    "time_since_exam",
    "case",
    "date_dx",
    "dx_time_after_exam",
    "dx_years_after_exam",
    "date_of_last_contact",
    "follow_up_time",
    "follow_up_years",
    "dicom_path",
    "png_path",
]
df_slim = df[columns_to_keep]
pd.set_option("display.max_colwidth", None)  # see max column width
len(df)

54535

In [5]:
df_slim.BreastImplantPresent.value_counts(dropna=False)
df_slim.BurnedInAnnotation.value_counts(dropna=False)
# just found out I didn't consider missing when filtering out based on breast implant present

NO     50834
YES     3701
Name: BurnedInAnnotation, dtype: int64

In [6]:
print(
    "filtering out {} dicoms with burned in annotations".format(
        len(df_slim[df_slim.BurnedInAnnotation != "NO"])
    )
)
print(
    "filtering out {} dicoms with breast implants".format(
        len(df_slim[df_slim.BreastImplantPresent == "YES"])
    )
)

filtering out 3701 dicoms with burned in annotations
filtering out 565 dicoms with breast implants


In [7]:
df_slim = df_slim[df_slim.BurnedInAnnotation == "NO"]  # filtering based on annotations
df_slim = df_slim[
    df_slim.BreastImplantPresent != "YES"
]  # filter out patients with breast implants

df_slim.exam_id.nunique()
len(df_slim)

50270

In [8]:
df_slim
# filter based on path existence:
df_slim["path_exists"] = df_slim["png_path"].apply(lambda x: os.path.isfile(x))
df_slim = df_slim[df_slim.path_exists == True]
len(df_slim)

33927

In [5]:
data_path = "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/CRDW_Registry_IndexDx_2020_Jul23.csv"
data = pd.read_csv(data_path)
data_path2 = "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/SPORE/CRDW_Registry_IndexDx_2021Mar18_update2021July27.dta"
data2 = pd.read_stata(data_path2)
pat = pd.DataFrame(data)


In [6]:
data2

,spore_id,mrn,IRB,dob,date_1stEnrol,sex,raceethnic,patient_first_name,patient_middle_name,patient_last_name,...,type1strecurrence,dateRec,typeRec,typeRec1,recur,year_recur,RFS,year_RFS,Allevent,year_event
0,,75716.0,16352A,1933-01-24,2008-11-24,2 Female,White,Dena,,Kentes,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,16.476385,0.0,16.476385,0.0,16.476385
1,,129163.0,16352A,1935-06-03,2009-07-07,2 Female,White,Patricia,,Jurkash,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,16.490076,1.0,16.490076,1.0,16.490076
2,,139661.0,16352A,1935-11-01,2009-04-17,2 Female,White,Mary,,Thorpe,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,11.958932,0.0,11.958932,0.0,11.958932
3,,140022.0,16352A,1909-09-21,2010-02-16,1 Male,White,Joseph,,Kirsner,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,3.252567,1.0,3.252567,1.0,3.252567
4,,234188.0,8962,1940-01-15,2005-05-20,2 Female,White,Roberta,,Siegel,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,14.286105,0.0,14.286105,0.0,14.286105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4582,,3996277.0,16352A,1971-02-13,2020-10-08,2 Female,White,Katherine,A,Jackson,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,0.459959,0.0,0.459959,0.0,0.459959
4583,,3997329.0,16352A,1959-05-20,2020-10-15,2 Female,White,Lucyna,,Cretu,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,0.522929,0.0,0.522929,0.0,0.522929
4584,,4007940.0,16352A,1936-04-27,2020-10-15,2 Female,White,Sharon,,Dillon,...,00 Patient became disease-free after treatment...,NaT,0.0,0.0,0.0,0.462697,0.0,0.462697,0.0,0.462697
4585,,4011707.0,16352A,1965-03-30,2020-10-14,2 Female,White,Marianne,,Tolar,...,"70 Since diagnosis, patient has never been dis...",NaT,70.0,70.0,NaN,NaN,0.0,0.509240,0.0,0.509240


In [10]:
total_n = len(data.index)
data = data[~pd.isnull(data.Age)]
print("filtered out {} entries without a valid age".format(total_n - len(data.index)))

filtered out 46 entries without a valid age


In [11]:
total_n = len(data.index)
data = data[~data.raceethnic.isin(["98", "Yes"])]

print(
    "filtered out {} entries without a valid race/ethnic entry".format(
        total_n - len(data.index)
    )
)

filtered out 2 entries without a valid race/ethnic entry


In [12]:
total_n = len(data.index)
data.mrn = pd.to_numeric(data.mrn, errors="coerce")
data = data[~pd.isnull(data.mrn)]
print("filtered out {} entries without a valid MRN".format(total_n - len(data.index)))

filtered out 0 entries without a valid MRN


In [13]:
total_n = len(data.index)
data = data[~pd.isnull(data.stage1)]
print(
    "filtered out {} entries without a valid stage at diagnosis".format(
        total_n - len(data.index)
    )
)

filtered out 85 entries without a valid stage at diagnosis


In [14]:
data.mrn = data.mrn.astype(int)
data.Age = data.Age.astype(int)
data.stage1 = data.stage1.astype(int)
data.her2 = pd.to_numeric(data.her2, errors="coerce")
data["datedx"] = pd.to_datetime(data["datedx"], infer_datetime_format=True)

In [15]:
data["binned_age"] = pd.cut(data.Age, bins=[0, 39, 49, 59, 69, data.Age.max()])
data["binned_age"].value_counts(sort=False)

(0, 39]      432
(39, 49]    1013
(49, 59]    1129
(59, 69]     964
(69, 99]     832
Name: binned_age, dtype: int64

In [16]:
race_groups = {
    "White": ["White"],
    "Black": ["Black"],
    "Asian / Pacific Islander": [
        "Asian Indian",
        "Asian Indian or Pakistani, NOS",
        "Filipino",
        "Chinese",
        "Other Asian, including Asian/Oriental, NOS",
        "Japanese",
        "Korean",
        "Hawaiian",
        "Vietnamese",
        "Thai (effective with 1994 diagnoses)",
        "Pakistani",
    ],
    "Other": ["Other", "Unknown"],
}

for label, group in race_groups.items():
    print(label, len(data[data.race_1.str.contains("|".join(group))]))

White 2439
Black 1680
Asian / Pacific Islander 175
Other 85


In [17]:
data.IRB.value_counts()

16352A          3068
8962, 16352A     696
8962             355
Pre-SPORE        251
Name: IRB, dtype: int64

In [18]:
mrn_to_study_id = pd.read_csv(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/mrn_to_study_id.csv",
    names=["mrn", "study_id"],
)
mirai_set = pd.read_csv(
    "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/mirai_chimec_all.csv"
)
downloaded_data = pd.read_pickle(
    "/gpfs/data/phs/groups/Projects/Huo_projects/SPORE/ojomoleye/data/downloaded_imaging_metadata.pkl"
)
val_results = pd.read_csv(
    "/gpfs/data/huo-lab/Image/ojomoleye/projects/mirai_validation/data/interim_full_validation_output.csv"
)

In [173]:
val_results["patient_id"] = val_results["patient_exam_id"].apply(
    lambda x: x.split('\t')[0])

In [174]:
#8 digit from radiology
#mrn to study id

,patient_exam_id,1_year_risk,2_year_risk,3_year_risk,4_year_risk,5_year_risk,patient_id
0,58982377\t2O10729,0.024250,0.036742,0.046101,0.054766,0.062453,58982377
1,91783139\t2O11253,0.001948,0.004567,0.007890,0.011327,0.014326,91783139
2,59441111\t2O10786,0.003628,0.008235,0.012976,0.016394,0.020359,59441111
3,59441111\t2O10787,0.004519,0.009567,0.014736,0.017790,0.021996,59441111
4,59441111\t2O10788,0.002534,0.006088,0.010050,0.014190,0.017754,59441111
...,...,...,...,...,...,...,...
1149,38461703\t2O07043,0.010150,0.019160,0.026575,0.031181,0.037213,38461703
1150,38461703\t2O07044,0.014565,0.023171,0.031222,0.036755,0.043330,38461703
1151,38461703\t2O07045,0.018022,0.028074,0.036731,0.043401,0.050489,38461703
1152,38461703\t2O07053,0.047988,0.068444,0.077761,0.093401,0.101330,38461703
